In [1]:
! pip install gdown

In [14]:
!pip install optuna

In [2]:
# Копируем файлы baseline.py, score.py, likes_data.zip
!gdown --id 1TsgFzNv58TzLy9QkgjKG68m1H79XGg7W
!gdown --id 1rG7ZX7wAgxKUSLZ_an4apis-UHEFCQXy
!gdown --id 1CqOrYqsj_wb4NKNBNF7Cc_cZ9WuF7T9Q

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1TsgFzNv58TzLy9QkgjKG68m1H79XGg7W
To: /kaggle/working/baseline.py
100%|██████████████████████████████████████| 2.37k/2.37k [00:00<00:00, 2.81MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1rG7ZX7wAgxKUSLZ_an4apis-UHEFCQXy
To: /kaggle/working/score.py
100%|██████████████████████████████████████████| 796/796 [00:00<00:00, 1.08MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't n

In [3]:
!unzip likes_data.zip

Archive:  likes_data.zip
replace test? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [4]:
# Импорт библиотек
import numpy as np
import gc
import os
import pandas as pd
import multiprocessing


from tqdm.notebook import tqdm
from scipy.sparse import csr_matrix, coo_matrix

In [5]:
# Всего 1.1 млн юзеров. Возьмем каждого 100 юзера (всего 10к юзеров)
# Возьмем последние 7 файлов, которые были выбраны юзерами. 
# Cамый последний файл будет таргетным значением

list_tracks = [] # список уникальных песен которые есть
df = [] # Список последних 10 лайкаанных песен юзера
y_list = [] # Последний лайк
with open('train') as f:

    lines = f.readlines()

    for line in tqdm(range(1, len(lines), 100)):

        row = [int(i) for i in lines[line].strip().split(' ')]

        #Игнорирую юзеров с менее 7 реакциями
        if len(row) < 8: continue

        tracks, y = row[-8:-1], row[-1]
        df.append(tracks)
        list_tracks.extend(tracks + [y])



  0%|          | 0/11601 [00:00<?, ?it/s]

In [6]:
list_tracks = set(list_tracks)

In [7]:
dict_with_tracks = {}

for i, el in tqdm(enumerate(list_tracks)):
    dict_with_tracks[el] = i

0it [00:00, ?it/s]

In [8]:
data = []
count_columns = len(list_tracks)

for row in tqdm(df):

    row_i = np.zeros(count_columns)
    filter_indices = [dict_with_tracks[i] for i in row]
    row_i[filter_indices] = 1
    data.append(row_i)

  0%|          | 0/11130 [00:00<?, ?it/s]

In [9]:
gc.collect()
del df, list_tracks, lines

In [10]:
df = pd.DataFrame(data)

In [11]:
df.to_csv('data_10k_user_7last_file.csv')

In [12]:
import pickle

f = open("dict_with_tracks.pkl","wb")
pickle.dump(dict_with_tracks, f)
f.close()

f = open("y_list.pkl","wb")
pickle.dump(y_list, f)
f.close()

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,44731,44732,44733,44734,44735,44736,44737,44738,44739,44740
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
matrix_coo = csr_matrix (df.values)
matrix_coo 


<11130x44741 sparse matrix of type '<class 'numpy.float64'>'
	with 77910 stored elements in Compressed Sparse Row format>